# Customizing PdfItDown

Once your app reaches a certain complexity, it might be good to customize it so that it can perform more complex and document-specific task that better adapt to your use case.

PdfItDown is designed to offer this flexibility, by allowing you to provide a custom `conversion_callback` to the `Converter` class constructor. 

The `conversion_callback`, tho, has to follow this function signature:

```python
def conversion_callback(
    input_path: str,
    output_path: str,
    title: str | None,
    overwrite: bool
)
```

While the name of the parameters can vary, their order should not be changed.

In [ ]:
# work in progress